# ***Linear Regression with PyTorch***

*Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*.

In [ ]:
import numpy as np
import torch

# **Training Data**

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

In [ ]:
# Target (apple, orange)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [ ]:
# Converting inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print()
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# ***Linear regression model from scratch***

In [ ]:
# weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.4289,  0.5188,  0.5510],
        [ 0.0255, -0.1042,  0.6054]], requires_grad=True)
tensor([-0.8120, -1.1038], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [ ]:
def model(x):
    return x @ w.t() + b

In [ ]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[26.3241, 19.8111],
        [41.0676, 30.7957],
        [63.3408, 22.2684],
        [-1.8717, 19.4204],
        [57.9604, 33.0323]], grad_fn=<AddBackward0>)


In [ ]:
#Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [ ]:
diff = preds - targets
diff * diff

torch.sum(diff * diff) / diff.numel()

tensor(3558.0183, grad_fn=<DivBackward0>)

# ***Loss Function***

In [ ]:
# MSE
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [ ]:
# Compute loss
loss = mse(preds, targets)
loss

tensor(3558.0183, grad_fn=<DivBackward0>)

# ***Compute Gradients***

In [ ]:
# Compute Gradients
loss.backward()

In [ ]:
# Gradient for weights
print(w)
print(w.grad)

tensor([[-0.4289,  0.5188,  0.5510],
        [ 0.0255, -0.1042,  0.6054]], requires_grad=True)
tensor([[-3237.0376, -3662.1914, -2219.1995],
        [-5482.1846, -6677.4990, -3948.3633]])


![postive-gradient](https://i.imgur.com/hFYoVgU.png)

![negative=gradient](https://i.imgur.com/w3Wii7C.png)

In [ ]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# ***Adjust weights and biases using gradient descent***

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [ ]:
# Generate perdictions
preds = model(inputs)
print(preds)

tensor([[26.3241, 19.8111],
        [41.0676, 30.7957],
        [63.3408, 22.2684],
        [-1.8717, 19.4204],
        [57.9604, 33.0323]], grad_fn=<AddBackward0>)


In [ ]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(3558.0183, grad_fn=<DivBackward0>)


In [ ]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-3237.0376, -3662.1914, -2219.1995],
        [-5482.1846, -6677.4990, -3948.3633]])
tensor([-38.8358, -66.9344])


In [ ]:
# Adjust weight & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [ ]:
print(w)
print(b)

tensor([[-0.3966,  0.5554,  0.5732],
        [ 0.0804, -0.0374,  0.6449]], requires_grad=True)
tensor([-0.8116, -1.1031], requires_grad=True)


In [ ]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(2473.8237, grad_fn=<DivBackward0>)


# ***Train for multiple epochs***

In [ ]:
for i in range(1000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [ ]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(0.4990, grad_fn=<DivBackward0>)


In [ ]:
preds

tensor([[ 57.0330,  70.1457],
        [ 82.2854, 100.7107],
        [118.7243, 133.1253],
        [ 21.1196,  37.1553],
        [101.8747, 118.9088]], grad_fn=<AddBackward0>)

In [ ]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

# ***Linear Regression using PyTorch***

In [ ]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 73], [91, 88, 64], [87, 134, 58],
                   [102, 43, 37], [69, 96, 70], [73, 67, 43],
                   [91, 88, 64], [87, 134, 58], [102, 43, 37],
                   [69, 96, 70], [73, 67, 73], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]],
                  dtype = 'float32')

In [ ]:
# Target (apple, orange)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

In [ ]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  73.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
# Define Data Loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [ ]:
for xb, yb in train_dl:
    print('batch : ')
    print(xb)
    print(yb)

batch : 
tensor([[ 73.,  67.,  73.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  73.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.]])
tensor([[ 56.,  70.],
        [103., 119.],
        [ 56.,  70.],
        [ 22.,  37.],
        [ 81., 101.]])
batch : 
tensor([[ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 22.,  37.],
        [119., 133.],
        [ 81., 101.],
        [103., 119.]])
batch : 
tensor([[ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 87., 134.,  58.]])
tensor([[119., 133.],
        [ 81., 101.],
        [103., 119.],
        [ 22.,  37.],
        [119., 133.]])


In [ ]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4635, -0.5605, -0.3331],
        [ 0.4615, -0.4978,  0.4645]], requires_grad=True)
Parameter containing:
tensor([-0.4049,  0.3074], requires_grad=True)


In [ ]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.4635, -0.5605, -0.3331],
         [ 0.4615, -0.4978,  0.4645]], requires_grad=True),
 Parameter containing:
 tensor([-0.4049,  0.3074], requires_grad=True)]

In [ ]:
preds = model(inputs)
preds

tensor([[ -96.1115,   34.5454],
        [-113.2271,   28.2173],
        [-135.1557,    0.6838],
        [ -84.1111,   43.1565],
        [-109.5116,   16.8686],
        [ -86.1183,   20.6118],
        [-113.2271,   28.2173],
        [-135.1557,    0.6838],
        [ -84.1111,   43.1565],
        [-109.5116,   16.8686],
        [ -96.1115,   34.5454],
        [-113.2271,   28.2173],
        [-135.1557,    0.6838],
        [ -84.1111,   43.1565],
        [-109.5116,   16.8686]], grad_fn=<AddmmBackward>)

In [ ]:
# Import nn.functional
import torch.nn.functional as F

In [ ]:
loss_fn = F.mse_loss

In [ ]:
loss = loss_fn(preds, targets)
loss

tensor(21582.2500, grad_fn=<MseLossBackward>)

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

# **Train the model**

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weight by subtracting a small quantity proportional to the gradient
5. Reset the gradient to zero

In [ ]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl) :

    # Repeat for given number of epochs
    for epoch in range(num_epochs):

        # Train with batch of data
        for xb, yb in train_dl:

            # 1.Generate predictions
            pred = model(xb)

            # 2.Calculate loss
            loss = loss_fn(pred, yb)

            # 3.Compute gradients
            loss.backward()

            # 4.Update parameter using gradients
            opt.step()

            # 5.Reset the gradients to zero
            opt.zero_grad()
             
        if (epoch + 1) % 10 == 0 :
            print('Epoch [{}/{}], Loss : {:.4f}'.format(epoch + 1, num_epochs, loss.item()))

In [ ]:
fit (100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss : 765.7924
Epoch [20/100], Loss : 790.1605
Epoch [30/100], Loss : 207.3519
Epoch [40/100], Loss : 431.5872
Epoch [50/100], Loss : 132.2032
Epoch [60/100], Loss : 152.3909
Epoch [70/100], Loss : 168.5138
Epoch [80/100], Loss : 56.3473
Epoch [90/100], Loss : 72.2837
Epoch [100/100], Loss : 80.9472


In [ ]:
preds = model(inputs)
preds

tensor([[ 66.8951,  89.0747],
        [ 79.1882,  97.5526],
        [119.7056, 129.2782],
        [ 27.8700,  45.2844],
        [ 92.9987, 109.7202],
        [ 57.6261,  70.3827],
        [ 79.1882,  97.5526],
        [119.7056, 129.2782],
        [ 27.8700,  45.2844],
        [ 92.9987, 109.7202],
        [ 66.8951,  89.0747],
        [ 79.1882,  97.5526],
        [119.7056, 129.2782],
        [ 27.8700,  45.2844],
        [ 92.9987, 109.7202]], grad_fn=<AddmmBackward>)

In [ ]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])